In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.applications.vgg16 import VGG16

c:\program files\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\program files\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\program files\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# help(VGG16)

In [3]:
vgg16 = VGG16(weights='imagenet',include_top=False,
              input_shape=(32, 32, 3))
# VGG16 模型在include_top=False时，可以自定义输入大小，至少32x32，通道必须是3
for layer in vgg16.layers[:15]:
    layer.trainable = False

addmodel = Sequential()
addmodel.add(Flatten(input_shape=vgg16.output_shape[1:]))
addmodel.add(Dense(256, activation='relu'))
addmodel.add(Dropout(0.3))
addmodel.add(Dense(10, activation='softmax'))

model = Model(inputs=vgg16.inputs, outputs=addmodel(vgg16.outputs))

In [4]:
train = pd.read_csv('train.csv')
y_train_full = train['label']
X_train_full = train.drop(['label'], axis=1)
X_test_full = pd.read_csv('test.csv')

In [5]:
X_train_full.shape

(42000, 784)

In [6]:
def process(data):
    data = np.array(data).reshape(-1,28,28) / 255.0
    output = np.zeros((data.shape[0], 32, 32, 3))
    for i in range(data.shape[0]):
        img = data[i]
        rgb_array = np.zeros((img.shape[0], img.shape[1], 3), "uint8")
        rgb_array[:, :, 0], rgb_array[:, :, 1], rgb_array[:, :, 2] = img, img, img
        pic = cv2.resize(rgb_array, (32, 32), interpolation=cv2.INTER_LINEAR)
        output[i] = pic
    return output

In [7]:
y_train = np_utils.to_categorical(y_train_full, 10)
X_train = process(X_train_full)
X_test = process(X_test_full)

print(X_train.shape)
print(X_test.shape)

(42000, 32, 32, 3)
(28000, 32, 32, 3)


In [8]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss="categorical_crossentropy",
              optimizer=opt, metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=1, batch_size=128,
                    validation_split=0.2)
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [1]:
model.summary()

NameError: name 'model' is not defined

In [2]:
from keras.utils import plot_model
plot_model(model, './model.png', show_shapes=True)

c:\program files\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\program files\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\program files\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


NameError: name 'model' is not defined

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
pred = model.predict(X_test)


In [ ]:
pred.reshape(X_test.shape[0],-1)
print(pred.shape)
image_id = pd.Series(range(1,len(pred)+1))
output = pd.DataFrame({'ImageId':image_id, 'Label':pred})
output.to_csv("submission_NN.csv",  index=False)